In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import re

: 

In [ ]:
def parse_recipe_steps_advanced(url, base_url):
    """
    Улучшенная версия парсера с обработкой разных вариантов разметки
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Ищем заголовок разными способами
        steps_header = soup.find('h2', string='Пошаговый рецепт')
        if not steps_header:
            steps_header = soup.find('h2', string=re.compile(r'Пошаговый', re.IGNORECASE))
        if not steps_header:
            steps_header = soup.find('h3', string=re.compile(r'Пошаговый', re.IGNORECASE))
        
        if not steps_header:
            print("Не найден заголовок с пошаговым рецептом")
            return None
        
        recipe_steps = []
        current_element = steps_header.find_next_sibling()
        
        while current_element:
            # Проверяем разные варианты заголовков шагов
            if (current_element.name in ['h4', 'h3', 'h5'] and 
                re.match(r'Шаг\s*\d+', current_element.get_text(), re.IGNORECASE)):
                
                step_number = current_element.get_text().strip()
                step_data = {
                    'step_number': step_number,
                    'step_index': len(recipe_steps) + 1,
                    'description': '',
                    'image_url': None
                }
                
                # Обрабатываем следующие элементы до следующего шага
                next_element = current_element.find_next_sibling()
                while (next_element and 
                       not (next_element.name in ['h4', 'h3', 'h5'] and 
                            re.match(r'Шаг\s*\d+', next_element.get_text(), re.IGNORECASE))):
                    
                    if next_element.name == 'p':
                        # Ищем изображение
                        img_tag = next_element.find('img')
                        if img_tag:
                            for attr in ['src', 'data-src', 'data-original']:
                                if img_tag.get(attr):
                                    step_data['image_url'] = base_url+img_tag[attr]
                                    break
                        
                        # Добавляем текст
                        text = next_element.get_text().strip()
                        if text and text != step_data['description']:
                            if step_data['description']:
                                step_data['description'] += ' ' + text
                            else:
                                step_data['description'] = text
                    
                    elif next_element.name == 'div' and next_element.find('img'):
                        # Обработка div с изображениями
                        img_tag = next_element.find('img')
                        for attr in ['src', 'data-src', 'data-original']:
                            if img_tag.get(attr):
                                step_data['image_url'] = img_tag[attr]
                                break
                    
                    next_element = next_element.find_next_sibling()
                    if not next_element:
                        break
                
                recipe_steps.append(step_data)
                current_element = next_element
            else:
                current_element = current_element.find_next_sibling()
        
        return recipe_steps
        
    except Exception as e:
        print(f"Ошибка: {e}")
        return None

: 

In [ ]:
def parse_ingredients_advanced(soup):
    """
    Парсит ингредиенты с различными стратегиями очистки
    """
    ingredients = []
    
    ingredients_header = soup.find(['h2', 'h3', 'h4'], 
                                 string=re.compile(r'Ингредиенты|Продукты|Состав', re.IGNORECASE))
    
    if ingredients_header:
        # Ищем в разных возможных местах
        containers = [
            ingredients_header.find_next('ul'),
            ingredients_header.find_next('ol'),
            ingredients_header.find_next('div', class_=re.compile(r'ingredient|product', re.IGNORECASE))
        ]
        
        for container in containers:
            if container:
                if container.name in ['ul', 'ol']:
                    for li in container.find_all('li'):
                        ingredient_data = parse_ingredient_item(li.get_text())
                        if(not ingredient_data.get("name")):
                            continue
                        ingredients.append(ingredient_data)
                break
    
    return ingredients

def parse_ingredient_item(raw_text):
    """
    Парсит отдельный элемент ингредиента, возвращает структурированные да
    нные
    """
    # Очищаем от лишних пробелов
    text = ' '.join(raw_text.strip().split())
    splitteed=raw_text.split()
    # Разделяем на название и количество
    parts = re.split(r'[\n\t]| {2,}', raw_text)
    name=""
    for part in parts:
        if part:
            name = part.strip()
            break
    if not parts: name = text.strip()
    # Пытаемся найти количество (если есть во второй части)
    amount = None
    unit = None
    
    if len(parts) > 1:
        # Ищем цифры и единицы измерения в оставшихся частях
        quantity_text = ' '.join(parts[1:])
        quantity_match = re.search(r'(\d+[\.,]?\d*)\s*([а-яa-z\.]+)?', quantity_text)
        if quantity_match:
            amount = quantity_match.group(1)
            unit = quantity_match.group(2)
    
    return {
        'name': name
    }


: 

In [ ]:
from urllib.parse import urljoin
def find_video_links_advanced(soup, base_url):
    """
    Расширенный поиск видео ссылок в различных местах
    """
    video_links = {
        'mp4': [],
        'youtube': [],
        'other': []
    }
    
    # Функция для добавления полного URL
    def add_full_url(url, type_key):
        if url and url not in video_links[type_key]:
            full_url = urljoin(base_url, url)
            video_links[type_key].append(full_url)
    
    # 1. Поиск в мета-тегах (Open Graph, Twitter Cards)
    meta_tags = soup.find_all('meta')
    for tag in meta_tags:
        property_name = tag.get('property', '').lower() or tag.get('name', '').lower()
        content = tag.get('content', '')
        if content.endswith('.mp4'):
                add_full_url(content, 'mp4')
        elif 'youtube' in content:
                video_links['youtube'].append(content)
    
    # 2. Поиск в тегах <video> и <source>
    video_tags = soup.find_all('video')
    for video in video_tags:
        # Атрибут src у самого video
        src = video.get('src')
        if src and src.endswith('.mp4'):
            add_full_url(src, 'mp4')
        
        # Теги source внутри video
        sources = video.find_all('source')
        for source in sources:
            src = source.get('src')
            type_attr = source.get('type', '')
            if src and (src.endswith('.mp4') or 'mp4' in type_attr):
                add_full_url(src, 'mp4')
    
    # 3. Поиск в тегах <a>
    youtube_patterns = [
        r'youtube\.com/watch\?v=([a-zA-Z0-9_-]+)',
        r'youtu\.be/([a-zA-Z0-9_-]+)',
        r'youtube\.com/embed/([a-zA-Z0-9_-]+)'
    ]
    
    all_links = soup.find_all('a', href=True)
    for link in all_links:
        href = link['href']
        
        # MP4 ссылки
        if href.endswith('.mp4'):
            add_full_url(href, 'mp4')
            continue
        
        # YouTube ссылки
        for pattern in youtube_patterns:
            match = re.search(pattern, href)
            if match:
                video_id = match.group(1)
                youtube_url = f'https://www.youtube.com/watch?v={video_id}'
                if youtube_url not in video_links['youtube']:
                    video_links['youtube'].append(youtube_url)
                break
    
    # 4. Поиск в iframe
    iframes = soup.find_all('iframe', src=True)
    for iframe in iframes:
        src = iframe['src']
        for pattern in youtube_patterns:
            match = re.search(pattern, src)
            if match:
                video_id = match.group(1)
                youtube_url = f'https://www.youtube.com/watch?v={video_id}'
                if youtube_url not in video_links['youtube']:
                    video_links['youtube'].append(youtube_url)
                break
    
    return video_links

def get_best_video_link(video_links):
    """
    Выбирает лучшую видео ссылку по приоритету
    """
    # Предпочтение: прямые MP4 ссылки
    if video_links['mp4']:
        return video_links['mp4'][0]  # Первая найденная MP4 ссылка
    
    # Затем YouTube
    if video_links['youtube']:
        return video_links['youtube'][0]  # Первая YouTube ссылка
    
    # Затем другие видео форматы
    if video_links['other']:
        return video_links['other'][0]
    
    return None

: 

In [ ]:
def find_ingredient_id_by_lemma(lemma: str, ingredients_file: str) -> int:
    """Находит ID ингредиента по лемме"""
    if not os.path.exists(ingredients_file):
        return None
    
    try:
        with open(ingredients_file, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f, delimiter='\t')
            for row in reader:
                if row.get('lemma', '').lower() == lemma.lower():
                    return int(row['id'])
    except Exception as e:
        print(f"Ошибка при поиске ингредиента: {e}")
    
    return None

: 

In [ ]:
def find_recipe_id_by_url(url: str, recipes_file: str) -> int:
    """Находит ID ингредиента по лемме"""
    if not os.path.exists(recipes_file):
        return None
    
    try:
        with open(recipes_file, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f, delimiter='\t')
            for row in reader:
                if row.get('url', '') == url:
                    return int(row['id'])
    except Exception as e:
        print(f"Ошибка при поиске рецепта: {e}")
    
    return None

: 

In [ ]:
import os
import json
import csv
from typing import List, Dict, Any

class IDManager:
    """Класс для управления ID между вызовами функций"""
    
    def __init__(self, state_file='id_state.json'):
        self.state_file = state_file
        self.state = self._load_state()
    
    def _load_state(self):
        """Загружает состояние ID из файла"""
        if os.path.exists(self.state_file):
            try:
                with open(self.state_file, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except:
                return {'recipe_id': 0, 'ingredient_id': 0, 'step_id': 0, 'video_id': 0}
        return {'recipe_id': 0, 'ingredient_id': 0, 'step_id': 0, 'video_id': 0}
    
    def _save_state(self):
        """Сохраняет состояние ID в файл"""
        with open(self.state_file, 'w', encoding='utf-8') as f:
            json.dump(self.state, f, ensure_ascii=False, indent=2)
    
    def get_next_recipe_id(self):
        """Возвращает следующий ID рецепта"""
        self.state['recipe_id'] += 1
        self._save_state()
        return self.state['recipe_id']
    
    def get_next_ingredient_id(self):
        """Возвращает следующий ID ингредиента"""
        self.state['ingredient_id'] += 1
        self._save_state()
        return self.state['ingredient_id']
    
    def get_next_step_id(self):
        """Возвращает следующий ID шага"""
        self.state['step_id'] += 1
        self._save_state()
        return self.state['step_id']
    
    def get_next_video_id(self):
        """Возвращает следующий ID видео"""
        self.state['video_id'] += 1
        self._save_state()
        return self.state['video_id']


: 

In [ ]:
%pip install pymorphy3

: 

In [ ]:
import pymorphy3

class IngredientManager:
    """Класс для управления ингредиентами с лемматизацией и проверкой уникальности"""
    
    def __init__(self, ingredients_file='ingredients.tsv'):
        self.ingredients_file = ingredients_file
        self.existing_ingredients = self._load_existing_ingredients()
        self.morph = pymorphy3.MorphAnalyzer()
    
    def _load_existing_ingredients(self) -> set[str]:
        """Загружает существующие леммы ингредиентов из файла"""
        existing_lemmas = set()
        
        if os.path.exists(self.ingredients_file):
            try:
                with open(self.ingredients_file, 'r', encoding='utf-8') as f:
                    reader = csv.DictReader(f, delimiter='\t')
                    for row in reader:
                        if 'lemma' in row and row['lemma']:
                            existing_lemmas.add(row['lemma'])
            except Exception as e:
                print(f"Ошибка при чтении файла ингредиентов: {e}")
        
        return existing_lemmas
    
    def get_lemma(self, ingredient_name: str) -> str:
        """
        Извлекает лемму главного слова из названия ингредиента
        """
        words = ingredient_name.split()
        if not words:
            return ingredient_name.lower()
        
        # Ищем главное слово (обычно последнее или существительное)
        main_word = words[-1]  # По умолчанию берем последнее слово
        
        # Пытаемся найти существительное
        for word in words:
            parsed = self.morph.parse(word)[0]
            if 'NOUN' in parsed.tag:
                main_word = word
                break
        
        # Лемматизируем главное слово
        parsed = self.morph.parse(main_word)[0]
        return parsed.normal_form.lower()
    
    def is_ingredient_unique(self, lemma: str) -> bool:
        """Проверяет, есть ли уже такой ингредиент"""
        return lemma not in self.existing_ingredients
    
    def add_ingredient_lemma(self, lemma: str):
        """Добавляет лемму в множество существующих"""
        self.existing_ingredients.add(lemma)

: 

In [ ]:
def json_to_tsv(original_data: Dict[str, Any], output_dir: str = ''):
    """
    Преобразует JSON данные в три TSV файла с проверкой уникальности ингредиентов
    """
    if (os.path.exists(f'{output_dir}recipes_raw.tsv') and find_recipe_id_by_url(original_data['url'], f'{output_dir}recipes_raw.tsv') is not None) or (os.path.exists(f'{output_dir}recipes.tsv') and find_recipe_id_by_url(original_data['url'], f'{output_dir}recipes.tsv') is not None):
        return
    # Инициализируем менеджеры
    id_manager = IDManager()
    ingredient_manager = IngredientManager(f'{output_dir}ingredients.tsv')
    
    # Получаем уникальные ID
    recipe_id = id_manager.get_next_recipe_id()
    
    # 1. Обрабатываем ингредиенты с проверкой уникальности
    ingredients_data = []
    ingredient_id_map = {}  # Для связи оригинального названия с ID
    
    for ingredient in original_data['ingredients']:
        ingredient_name = ingredient['name']
        lemma = ingredient_manager.get_lemma(ingredient_name)
        
        # Проверяем, есть ли уже такой ингредиент
        if ingredient_manager.is_ingredient_unique(lemma):
            # Новый уникальный ингредиент
            ingredient_id = id_manager.get_next_ingredient_id()
            ingredients_data.append({
                'id': ingredient_id,
                'name': ingredient_name,
                'lemma': lemma
            })
            ingredient_id_map[ingredient_name] = ingredient_id
            ingredient_manager.add_ingredient_lemma(lemma)
        else:
            # Ингредиент уже существует, находим его ID
            # Для этого нужно прочитать существующие данные
            existing_id = find_ingredient_id_by_lemma(lemma, f'{output_dir}ingredients.tsv')
            if existing_id:
                ingredient_id_map[ingredient_name] = existing_id
    
    # 2. Создаем TSV для шагов
    steps_data = []
    for step in original_data['steps']:
        step_id = id_manager.get_next_step_id()
        steps_data.append({
            'id': step_id,
            'recipe_id': recipe_id,
            'step_number': step['step_number'],
            'step_index': step['step_index'],
            'description': step['description'],
            'image_url': step['image_url'],
        })
    
    # 3. Создаем TSV для рецепта
    video_id = id_manager.get_next_video_id()
    
    # Получаем ID ингредиентов для этого рецепта
    recipe_ingredient_ids = []
    for ingredient in original_data['ingredients']:
        ingredient_name = ingredient['name']
        if ingredient_name in ingredient_id_map:
            recipe_ingredient_ids.append(str(ingredient_id_map[ingredient_name]))
    
    recipe_data = [{
        'id': recipe_id,
        'title': '*',
        'url': original_data['url'],
        'ingredient_ids': '|'.join(recipe_ingredient_ids),
        'step_ids': '|'.join(str(step['id']) for step in steps_data),
        'video_id': video_id,
        'total_steps': original_data['total_steps'],
        'total_ingredients': len(recipe_ingredient_ids),
        'mp4_url': original_data['video']['mp4'][0] if original_data['video']['mp4'] else '',
        'youtube_url': original_data['video']['youtube'][0] if original_data['video']['youtube'] else ''
    }]
    
    # Сохраняем в TSV файлы
    save_to_tsv(ingredients_data, f'{output_dir}ingredients.tsv', ['id', 'name', 'lemma'])
    save_to_tsv(steps_data, f'{output_dir}steps_raw.tsv', ['id', 'recipe_id', 'step_number', 'step_index', 'description', 'image_url'])
    save_to_tsv(recipe_data, f'{output_dir}recipes_raw.tsv', ['id', 'title', 'url', 'ingredient_ids', 'step_ids', 'video_id', 'total_steps', 'total_ingredients', 'mp4_url', 'youtube_url'])
    
def save_to_tsv(data: List[Dict], filename: str, fieldnames: List[str]):
    """
    Сохраняет данные в TSV файл
    """
    need_header = not os.path.exists(filename)
    with open(filename, 'a', encoding='utf-8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames, delimiter='\t')
        if need_header:
            writer.writeheader()
        writer.writerows(data)

: 

In [ ]:
urls = ['https://eda.video/uzbekskaia-dymliama', 'https://eda.video/losos-v-slivocnom-souse-na-skovorode']
base_url="https://eda.video"
if not os.path.isdir('./data'):
    os.mkdir('./data')
for url in urls:
    try:
        # Парсим шаги
        steps = parse_recipe_steps_advanced(url, base_url)
        if steps:
                print(f"Найдено шагов: {len(steps)}")
                
                # Парсим ингредиенты
                response = requests.get(url)
                soup = BeautifulSoup(response.text, 'html.parser')
                ingredients = parse_ingredients_advanced(soup)
                video_url=find_video_links_advanced(soup, base_url)
                # Создаем полную структуру рецепта
                recipe_data = {
                    'url': url,
                    'ingredients': ingredients,
                    'steps': steps,
                    'total_steps': len(steps),
                    'video':video_url
                }
                
                json_to_tsv(recipe_data, "./data/")
                print("Данные сохранены в папку data")
        else:
                raise Exception("Анлуко")
    except Exception as e:
        print(f"Не удалось распарсить рецепт: {url}, потому что {str(e)}")

: 